In [1]:
import numpy as np
import utilities as ut
import gizmo_analysis as gizmo

In [2]:
def getAngularMomentum(pos, dist, vel, radius = 10):
    
    mask = dist < radius
    pos_sel = pos[mask]
    
    # L = r x v (direction invariant of mass)
    L_vector = np.median(np.cross(pos, vel), axis=0)
    L_hat = L_vector / np.linalg.norm(L_vector) # normalize
    return L_vector

In [28]:
def getSymmetryAxes(pos, dist, radius):
    print('Assigning inertia tensor at r = ', radius, 'kpc')
    mask = dist < radius
    pos = pos[mask]
    
    # Get moment of inertia tensor
    xx = np.sum(pos[:, 0]**2 / (pos[:, 0]**2 + pos[:, 1]**2 + pos[:, 2]**2))
    yy = np.sum(pos[:, 1]**2 / (pos[:, 0]**2 + pos[:, 1]**2 + pos[:, 2]**2)) 
    zz = np.sum(pos[:, 2]**2 / (pos[:, 0]**2 + pos[:, 1]**2 + pos[:, 2]**2))
    xy = yx = np.sum(pos[:, 0]*pos[:, 1] / (pos[:, 0]**2 + pos[:, 1]**2 + pos[:, 2]**2))
    xz = zx = np.sum(pos[:, 0]*pos[:, 2] / (pos[:, 0]**2 + pos[:, 1]**2 + pos[:, 2]**2))
    zy = yz = np.sum(pos[:, 2]*pos[:, 1] / (pos[:, 0]**2 + pos[:, 1]**2 + pos[:, 2]**2))
    
    Im = [[xx, xy, xz], [yx, yy, yz], [zx, zy, zz]]
    
    e_val, e_vec = np.linalg.eig(Im)
    sorted_idx = np.argsort(e_val)[::-1] # sort from largest to smallest (idx of 2 is minor axis)
    sorted_val = e_val[sorted_idx]
    sorted_vec = np.transpose(e_vec)[sorted_idx]
    
    # Enforce RHR onto an axis
#     sorted_vec[2] = np.cross(sorted_vec[0], sorted_vec[1])
    
    return sorted_vec

In [4]:
def getMinAngle(vec, ref):
    # ref is the reference vector
    # vec is the vector of interest
    
    vec = vec/np.linalg.norm(vec)
    ref = ref/np.linalg.norm(ref)
    
    angle = np.min(
        [np.arccos(np.dot(-1*ref, vec)),
        np.arccos(np.dot(ref, vec))]
    )
    
    return angle

In [5]:
sim_dir = '../data/latte_metaldiff/m12i_res7100'
part = gizmo.io.Read.read_snapshots(['dark', 'star'], 'redshift', 0, sim_dir)


# in utilities.simulation.Snapshot():
* reading:  data/latte_metaldiff/m12i_res7100/snapshot_times.txt

* input redshift = 0:  using snapshot index = 600, redshift = 0.000


# in gizmo_analysis.gizmo_io.Read():
* reading header from:  data/latte_metaldiff/m12i_res7100/output/snapdir_600/snapshot_600.0.hdf5
  snapshot contains the following number of particles:
    dark      (id = 1): 70514272 particles
    dark2     (id = 2): 5513331 particles
    gas       (id = 0): 57060074 particles
    star      (id = 4): 13976485 particles
    blackhole (id = 5): 0 particles

* reading species: ['dark', 'star']
* reading particles from:
    snapshot_600.0.hdf5
    snapshot_600.1.hdf5
    snapshot_600.2.hdf5
    snapshot_600.3.hdf5

* reading cosmological parameters from:  data/latte_metaldiff/m12i_res7100/initial_condition/ic_agora_m12i.conf

* checking sanity of particle properties


# in gizmo_analysis.gizmo_track.ParticleCoordinate():
  read 1 host (position, velocity, principal axes) from:  d

In [6]:
positions = part['dark'].prop('host.distance')
dists = part['dark'].prop('host.distance.total')

In [7]:
pos_star = part['star'].prop('host.distance')
vel_star = part['star'].prop('host.velocity')
dist_star = part['star'].prop('host.distance.total')

l_vec = getAngularMomentum(pos_star, dist_star, vel_star, radius=20)

In [29]:
symm_axs = []
for i in np.arange(10, 60, step = 10):
    symm_axs.append(getSymmetryAxes(positions, dists, radius = i))

R:  10
R:  20
R:  30
R:  40
R:  50


In [38]:
min_ax_host = part.host['rotation'][0, 2]

In [45]:
angle = []
for tensor in symm_axs:
    angle.append(getMinAngle(tensor[2], min_ax_host) * 180/np.pi)